# Libraries

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score, confusion_matrix, roc_auc_score, roc_curve # this is used to evaluate the model.
from sklearn.model_selection import train_test_split # to separate the dataset.
from sklearn.feature_extraction.text import TfidfVectorizer #used for the transformation of text data.
from sklearn.tree import DecisionTreeClassifier
import time #pour étudier le temps d'execution d'entrainement, de notre meilleure modèle.
import psutil #pour calculer la mémoire.
import os #pour calculer le temps d'execution
import pickle as pkl #file to save and load a model.

# Data access

In [12]:
data=pd.read_csv('title_text.csv').drop(['Unnamed: 0'],axis=1).iloc[:,:150]
data.head()

,title_processed,text_processed,isFake
0,white hous threaten veto senat defens bill,washington reuter white hous threaten tuesday ...,False
1,colleg socialist group incit child yell kill d...,knight social hold event allow student bash pi...,True
2,kuwait could pay parti trump hotel washington,correct feb stori show event take place wednes...,False
3,professor claim sexual harass ban violat first...,group colleg professor releas report claim fed...,True
4,trump say role we libya defeat islam state,washington reuter presid donald trump say thur...,False


## Modification du text en données numériques.
Nous partons du principe que les données ont étés clean et que nous pouvons nous concerntrer à créer un model.

Comme les informations principales pour définir des Fake news et des vrai news, se base sur le texte et le titre, nous devons nous focaliser sur ces informations. Il faut ainsi que nous transformons le text et le titre en données afin que notre model puisse utiliser des données numériques pour trouver la bonne réponse. Nous allons utiliser la methode de TF-IDF vectorizing.

### TF-IDF vectorizer
Cette méthode offre la possibilité de déterminer les scores TF-IDF de nos données textuelles. Ce qui signifie : 

TF -> la Fréquence du Terme : le nombre de fois qu'un mot apparaît, en éliminant les mots qui ne sont pas importants.

IDF -> la fréquence inverse de détection : le même concept que TF, mais l'analyse va au-delà de la simple ligne, la fréquence dans l'ensemble du dataset est analysée.

Ces informations seront utilisées comme données pour notre modèle d'apprentissage automatique.

(il est important de faire cela uniquement sur nos données d'entraînement, sinon le modèle connaîtra les données de test)

In [13]:
data.columns

Index(['title_processed', 'text_processed', 'isFake'], dtype='object')

In [15]:
X=data.drop(['isFake'],axis=1)
y=data['isFake']
#spearation en données d'entrainement et de test.
X_train, X_test, y_train, y_test = train_test_split(X['text_processed'], y, test_size=0.3, random_state=42)

tfidf_vectorizer=TfidfVectorizer(use_idf=True)

# On fit et transforme les données d'entrainement.
tfidf_train=tfidf_vectorizer.fit_transform(X_train).toarray()

# on transfomer uniquement le test set, sinon il va connaitre les données test...
tfidf_test=tfidf_vectorizer.transform(X_test)

MemoryError: Unable to allocate 27.5 GiB for an array with shape (29116, 126692) and data type float64

# model decision tree
Un arbre de décision est une technique d'apprentissage supervisé utilisée pour résoudre des problèmes de classification et de régression. Il vise à modéliser la relation entre une variable cible, qui dans notre cas sont les Fake news, et plusieurs variables explicatives, dans ce cas nous utiliserons le texte de la news en question.

L'approche de l'arbre de décision suit la stratégie de **"diviser pour conquérir"**.

Cet algorithme itératif construit progressivement l'arbre. Au départ, il sélectionne l'attribut qui divise le mieux les données, formant des groupes en relation avec la variable cible. À chaque étape suivante, il répète ce processus pour chaque sous-ensemble créé précédemment, en se basant sur les nœuds déjà formés, jusqu'à ce que les critères de terminaison soient satisfaits (ex: le nombre maximal de niveaux dans l'arbre)

Une fois l'arbre construit, il peut être utilisé pour prédire l'issue pour de nouvelles données en les faisant passer à travers l'arbre, de la racine jusqu'à une feuille, qui représente la décision finale ou la prédiction de la variable cible.

src: https://scikit-learn.org/stable/modules/tree.html

In [4]:
#nous construisons le modèle
clf = DecisionTreeClassifier(max_depth=1,random_state=42)
#nous l'entrainons
clf.fit(tfidf_train, y_train)

#nous obtenons une accuracy du modèle
Accuracy = clf.score(tfidf_test, y_test)
print(Accuracy*100)

100.0


# Optimisation
Pour optimiser le model nous allons modifier les paramètres suivants:
- **criterion**: Comme son nom l'indique c'est le critère de qualité que notre modèle utilise pour séparer les données et de créer les noeuds. Nous allons optimiser le modèle avec les valeurs suivantes: **[“gini”, “entropy”, “log_loss”]**
- **max_depth**: la profondeur maximale que nous autorisons à notre arbre. Nous allons optimiser le modèle avec les valeurs suivantes: **[10,100,1000,None]**
- **splitter**: C'est la stratégie que notre modèle utilise pour créer les noeuds. Nous allons optimiser le modèle avec les valeurs suivantes: **[“best”, “random”]**

In [5]:
params_grid={'criterion':["gini", "entropy", "log_loss"],
             'max_depth':[5,10],
             'splitter': ["best", "random"]
            }

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import make_scorer, accuracy_score


# Initialize the Decision tree classifier
model = DecisionTreeClassifier(random_state=42)

# On creer un scorer pour le grid search 
scorer = make_scorer(accuracy_score)

# Initialisation de la GridSearch pour trouver le meilleur C.
grid_search = GridSearchCV(estimator=model, param_grid=params_grid, scoring=scorer, cv=5)

# On lance les multiples entrainements.
grid_search.fit(tfidf_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [5, 10], 'splitter': ['best', 'random']},
             scoring=make_scorer(accuracy_score))

In [1]:
# Get the best combination of parameters
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best score: {best_score}")

NameError: name 'grid_search' is not defined

# Model Evaluation
Afin d'évaluer le modèle nous faisons une représentation graphique des résultats obtenus. Nous allons étudier les metrics suivants:
- le **temps d'entrainement** que le modèle a besoin, ainsi que son **taux de mémoire (Mo)**.
- **matrice de confusion**, qui est un excellent choix d'évaluation de performance, car nous sommes dans le cas d'une classification binaire. 
- **L'accuracy** du model.
- **La précision**
- **recall**
- **F1-score**.
- **ROC-AUC score**

src: https://www.v7labs.com/blog/performance-metrics-in-machine-learning#h2

In [9]:
#creation du modèle avec les meilleurs paramètres
best_model=DecisionTreeClassifier(#C=best_params['criterion'],
                                  max_depth=best_params['max_depth'],
                                  splitter=best_params['splitter'],
                                  random_state=42)


Nous voullons étudier **le temps d'entrainement (en h/m/s)** et **l'utilisation de la mémoire (Mo)** que le modèle a besoin.

In [11]:
def second_to_hms(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return int(hours), int(minutes), int(seconds)

def bytes_to_Mo(mem_bytes):
    mem_kb = mem_bytes / 1024  # Convertir en kilooctets
    mem_mb = mem_kb / 1024  # Convertir en mégaoctets
    return mem_mb

#lancemenet de l'enrestristrement de la mémoire.
process = psutil.Process(os.getpid())
mem_before_bytes = process.memory_info().rss

#lancement de l'enregistrement du temps d'entrainement.
start_time = time.time() 
# entrainement du modèle.
best_model.fit(tfidf_train,y_train)
end_time = time.time()

#le temps écoulé 
elapsed_time= end_time - start_time

print(f"Temps d'entrainement (h | m | s) : {second_to_hms(elapsed_time)[0]} | {second_to_hms(elapsed_time)[1]} | {second_to_hms(elapsed_time)[2]}")

mem_after_bytes = process.memory_info().rss
#on convertit les bytes en Mo.
mem_bytes=mem_after_bytes - mem_before_bytes


print(f"Utilisation de la mémoire (Mo) : {bytes_to_Mo(mem_bytes)}")
#Create or open a file with write-binary mode and save the model to it
pickle = pkl.dump(model, open('DecisionTree_model.pickle', 'wb'))

Temps d'entrainement (h | m | s) : 0 | 0 | 2
Utilisation de la mémoire (Mo) : 0.00390625


In [17]:
#optention des prédictions de ce modèle.
best_model = pkl.load(open('DecisionTree_model.pickle', 'rb'))
y_pred=model.predict(tfidf_test)
y_pred=best_model.predict(tfidf_test)

NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
#creation de la matrice de confusion
cm = confusion_matrix(y_test, y_pred)

#représentation graphique du résultat du meilleure model
sns.heatmap(cm, annot=True)
plt.xlabel('Les labels prédits')
plt.ylabel('Les vrais labels')
plt.title('Matrice de confusion')
plt.show()

print(f"le modèle a donc {cm[0][0]} instances vrai Positives et {cm[1][1]} instances de Vrai négatives, {cm[1][0]} instances de Faux Positives et {cm[0][1]} instances de Faux négatives. ")



In [ ]:
acc=accuracy_score(y_pred,y_test)
print(f"Notre modèle a une précision globale de {round(acc*100,2)}%, ce qui signifie qu'il prédit correctement les classes des instances dans {round(acc*100,2)}% des cas.")

In [ ]:
precision=precision_score(y_test,y_pred)
print(f"notre model a une precision de {round(precision*100,2)}% . lorsqu'il prédit une classe comme positive, il a raison dans {round(precision*100,2)}% des cas.")

In [ ]:
recall=recall_score(y_test,y_pred)
print(f"notre model arrive a détecter {round(recall*100,2)}%. Donc sur l'ensemble des vrai positives, le modèle parvient à en identifier correctement {round(recall*100,2)}% des cas.")

In [ ]:
f1score=f1_score(y_test,y_pred)
print(f"notre model a un F1-score de {round(f1score*100,2)}%, ce qui signifie que il y a un excellent equilibre entre le recall et la précision.")

In [ ]:
roc_auc=roc_auc_score(y_test,y_pred)
print(f"Le modèle a donc une score ROC-AUC de {roc_auc} ce qui est très proche de 1, donc le modèle a une excellente capacité à différencier entres les Fake et les vrais news.")

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('Taux de Faux Positives (%)')
plt.ylabel('Taux de Vrai Positives (%)')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

## Evaluation du vrAI 

In [ ]:
vrAI = 0
acc = accuracy_score(y_pred,y_test)
if acc > 0.90:
    vrAI = 'Fx'
elif  0.70<acc<=0.90:
    vrAI = 'F'
elif  0.50<acc<=0.70:
    vrAI = 'E'
elif  0.30<acc<=0.50:
    vrAI = 'D'
elif  0.15<acc<=0.30:
    vrAI = 'C'
elif  0.05<acc<=0.15:
    vrAI = 'B'
else:
    vrAI = 'A'